In [1]:
from qutip import *
from numpy import pi, linalg
from scipy.special import jv,j0
from math import sqrt
import numpy as np

In [2]:
si = qeye(2)

sx = sigmax()

sy = sigmay()

sz = sigmaz()

n = 1

m = 1

extmagfield_m = Qobj([[0, 0, 0],[0, 0, 0],[1, 1, 1]])

extmagfield_n = Qobj([[0, 0, 0],[0, 0, 0],[0.3, 0.3, 0.3]])

JJ = Qobj([[0, 0, 0], [0, 0, 0], [0, 0, 0]])

AA = Qobj([[0, 0, 0] ,[0, 0, 0], [0.99925, 0.625147, 0.551273]])
gama = Qobj([[0.0124425, 0.0806628, 0.00999575],[0.0550028, 0.0758354, 0.07346340],[0.0972069, 0.0723954, 0.07405450]])
# gama = 1
t = 3
E1 = (
    1 / 2 * linalg.norm(JJ)
    + 1 / 2 * linalg.norm(gama)
    + 1 / 2 * linalg.norm(AA)
    + linalg.norm(extmagfield_n)
    + linalg.norm(extmagfield_m)
)
tau = E1 * t / 2
print(E1)
kappa = 3 / 2 * tau
print(tau)

3.0036597693272094
4.505489653990814


In [3]:
def spin_op(N):
    """calculates spin oprators

    Args:
        N (int): number of electrons or atoms

    Returns:
        matrix: matrices of spin oprators
    """
    Sx = []
    Sy = []
    Sz = []

    for n in range(N):
        x_list = []
        y_list = []
        z_list = []

        for m in range(N):
            x_list.append(si)
            y_list.append(si)
            z_list.append(si)

        x_list[n] = sx
        y_list[n] = sy
        z_list[n] = sz

        Sx.append(tensor(x_list))
        Sy.append(tensor(y_list))
        Sz.append(tensor(z_list))

    return [Sx, Sy, Sz]

In [4]:
Sx, Sy, Sz = spin_op(m+n)
Sx

[Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
 Qobj data =
 [[0. 0. 1. 0.]
  [0. 0. 0. 1.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]],
 Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
 Qobj data =
 [[0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 0. 1.]
  [0. 0. 1. 0.]]]

In [5]:
Sx, Sy, Sz = spin_op(m+n)
# HS & HB are bare hamiltonians of the central system and bath
# V is the system-bath interaction
HS = HB = V = 0
for i in range(m):
    HS += extmagfield_m[0,i] * Sx[i] + extmagfield_m[1,i] * Sy[i] + extmagfield_m[2,i] * Sz[i]
    for j in range(m):
        HS += JJ[0,j] * Sx[i] * Sx[j] + JJ[1,j] * Sy[i] * Sy[j] + JJ[2,j] * Sz[i] * Sz[j]

for i in range(m,m+n):
    HB += extmagfield_n[0,i] * Sx[i] + extmagfield_n[1,i] * Sy[i] + extmagfield_n[2,i] * Sz[i]
    for j in range(m,m+n):
        HB += gama[0,j] * Sx[i] * Sx[j] + gama[1,j] * Sy[i] * Sy[j] + gama[2,j] * Sz[i] * Sz[j]
for i in range(m,m+n):
    for j in range(m):
        V += AA[0,j] * Sx[i] * Sx[j] + AA[1,j] * Sy[i] * Sy[j] + AA[2,j] * Sz[i] * Sz[j]
HH = HS + HB + V


In [6]:
HH

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 2.5281436  0.         0.         0.       ]
 [ 0.        -0.0703564  0.         0.       ]
 [ 0.         0.        -1.4703564  0.       ]
 [ 0.         0.         0.        -0.0718564]]

In [7]:
def TG(k, G):
    if k == 0:
        return 1
    elif k == 1:
        return G
    else:
        return 2 * G * TG(k-1, G) - TG(k-2, G)

In [8]:
G = HH / E1
UU = 0
for k in range(int(kappa) + 1):
    a = 1
    if k == 0:
        a = 2
    UU += a * ((-1j) ** k) * jv(k,tau) * TG(k, G)



In [9]:
UU

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[-0.87492589+0.32364412j  0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.00868537+0.04718951j  0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j         -0.77029291+0.41797466j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   0.00857325+0.04819242j]]

In [10]:
zero = basis(2) 
c = create(2)
one=c*zero
psi=tensor(tensor(one,basis(2)))
psi0=psi*psi.dag()
print("trace: {}".format(psi0.tr())) 
psi0

trace: 1.0


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]]

In [11]:
psif = UU * psi0
result = psif * psif.dag()
print("trace: {:.2f}".format(result.tr()))
result
# psif = UU * psi0 * UU.dag()
# print("trace: {:.2f}".format(psif.tr()))
# psif

trace: 0.77


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.76805399 0.        ]
 [0.         0.         0.         0.        ]]

In [12]:
UU*UU.dag()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.87024083 0.         0.         0.        ]
 [0.         0.00230229 0.         0.        ]
 [0.         0.         0.76805399 0.        ]
 [0.         0.         0.         0.00239601]]